In [100]:
# Initialize Mistral client with API key
from mistralai import Mistral
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.environ["MISTRAL_KEY"] # Replace with your API key
client = Mistral(api_key=api_key)

# Import required libraries
from pathlib import Path
from mistralai import DocumentURLChunk, ImageURLChunk, TextChunk
import json

# Verify PDF file exists
import base64

# Verify image exists
 
image_file = Path("/Users/eliot/Desktop/project_ML/project_ML_Facturation/dataset/receipts/1004-receipt.jpg")
assert image_file.is_file()

# Encode image as base64 for API
encoded = base64.b64encode(image_file.read_bytes()).decode()
base64_data_url = f"data:image/jpeg;base64,{encoded}"

image_response = client.ocr.process(
    document=ImageURLChunk(image_url=base64_data_url),
    model="mistral-ocr-latest"
)

# Convert response to JSON
response_dict = json.loads(image_response.model_dump_json())
json_string = json.dumps(response_dict, indent=4)

image_ocr_markdown = response_dict["pages"][0]["markdown"]
# Get structured response from model
chat_response = client.chat.complete(
    model="pixtral-12b-latest",
    messages=[
        {
            "role": "user",
            "content": [
                ImageURLChunk(image_url=base64_data_url),
                TextChunk(
                    text=(
                        f"This is image's OCR in markdown:\n\n{image_ocr_markdown}\n.\n"
                        "I wan't same structured json with only 'name' restaurant final 'total' and 'date', and the 'adress'"
                        "The output should be strictly be json with no extra commentary"
                        
                    )
                ),
            ],
        }
    ],
    response_format={"type": "json_object"},
    temperature=0,
)

# Parse and return JSON response
response_dict = json.loads(chat_response.choices[0].message.content)



In [112]:
combined_df

,date,amount,currency,vendor
0,2016-09-01,7.61,USD,"7230 Pendleton Pike, Indianapolis, IN 46226"
1,2018-07-21,25.78,USD,"889 W. 190th Street, Los Angeles, CA 90248"
2,2014-12-21,25.94,USD,"Hialeah, FL 33010"
3,2017-05-12,45.58,USD,Ortega Pizzeria & Tavern
4,2016-11-12,117.00,USD,JBR
...,...,...,...,...
195,NaN,24.47,USD,Dona Mercedes Restaurant
196,NaN,9.58,USD,"11732 ARTESIA BLVD. ARTESIA, CA."
197,NaN,40.30,USD,"18128 Pioneer Blvd, Artesia, CA 90701"
198,NaN,10.23,USD,"Golden Coin, 94-450 Ukee St, Waipahu, HI 96797"


In [5]:

import os
import pandas as pd

folder_path = "/Users/eliot/Desktop/project_ML/project_ML_Facturation/dataset/bank_statements"
all_dfs = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # You can change this for other file types
        full_path = os.path.join(folder_path, filename)
        df = pd.read_csv(full_path)
        all_dfs.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(all_dfs, ignore_index=True)





In [107]:
combined_df = combined_df.dropna(subset=["vendor"])
col1  = response_dict["name"] 
#+' '+ response_dict["address"] 

In [122]:
response_dict["total"]

15.03

In [140]:
combined_df

,date,amount,currency,vendor
0,2016-09-01,7.61,USD,"7230 Pendleton Pike, Indianapolis, IN 46226"
1,2018-07-21,25.78,USD,"889 W. 190th Street, Los Angeles, CA 90248"
2,2014-12-21,25.94,USD,"Hialeah, FL 33010"
3,2017-05-12,45.58,USD,Ortega Pizzeria & Tavern
4,2016-11-12,117.00,USD,JBR
...,...,...,...,...
195,NaN,24.47,USD,Dona Mercedes Restaurant
196,NaN,9.58,USD,"11732 ARTESIA BLVD. ARTESIA, CA."
197,NaN,40.30,USD,"18128 Pioneer Blvd, Artesia, CA 90701"
198,NaN,10.23,USD,"Golden Coin, 94-450 Ukee St, Waipahu, HI 96797"


In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Remove rows with NaN in 'vendor' column from the DataFrame itself
combined_df = combined_df.dropna(subset=["vendor"])
col1  = response_dict["name"] +' '+ response_dict["address"]  # your target string
# Then proceed with your code
embeddings_col1 = "GOLDEN BOWL TERIYAKI"
embeddings_col2 = combined_df["vendor"].tolist()  # list of vendors (cleaned)

# Create a combined list for TF-IDF (target first, then all vendors)
all_texts = [embeddings_col1] + embeddings_col2

# Vectorize
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_texts)

# Compute cosine similarity between restaurant_address and all vendors
similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

# Put the results in a DataFrame
similarity_df = pd.DataFrame({
    "vendor": embeddings_col2,
    "similarity": similarities,
    "amount":response_dict["total"]
}).sort_values(by="similarity", ascending=False).head(5)

# Sort by similarity

# Display top matches


In [142]:
similarity_df.join()


,vendor,similarity,amount
106,GOLDEN BOWL TERIYAKI,1.000000,15.03
50,Golden Panda Buffet,0.265332,15.03
169,The Golden Spear Cafe,0.241854,15.03
196,"Golden Coin, 94-450 Ukee St, Waipahu, HI 96797",0.149868,15.03
2,"Hialeah, FL 33010",0.000000,15.03


In [79]:
from simple_elmo import ElmoModel
model = ElmoModel()
model.load("/content/209.zip")
 
sentence = "After stealing gold from the bank vault, the bank robber was seen fishing on the river bank."

elmo_vectors = model.get_elmo_vectors(sentence, layers="average")
print(f"Tensor shape: {elmo_vectors.shape}")



ModuleNotFoundError: No module named 'tensorflow'

In [64]:
restaurant_adress.split(" ")

['Akira', '385', '5th', 'Ave,', 'NY', '10018']

In [77]:
tfidf_matrix.vector

AttributeError: 'csr_matrix' object has no attribute 'vector'

In [81]:
import spacy
nlp = spacy.load('en_core_web_md')
docs = [nlp(headline) for headline in restaurant_adress.split(" ")]

In [93]:
import spacy
import pandas as pd

# Load medium model with vectors
nlp = spacy.load('en_core_web_md')

# Your column of text (e.g. vendor names or headlines)
headlines = combined_df["vendor"].dropna().astype(str).tolist()

# Text to compare
target_text = "8105 MADISON AVE	"  # should be a single string

# Convert all to spaCy docs
target_doc = nlp(target_text)
doc_list = [nlp(text) for text in headlines]

# Compute similarity scores
similarities = [target_doc.similarity(doc) for doc in doc_list]

# Create a DataFrame with results
similarity_df = pd.DataFrame({
    "vendor": headlines,
    "similarity": similarities
})

# Sort by similarity
similarity_df.sort_values(by="similarity", ascending=False, inplace=True)

# Show top results



/var/folders/n3/b6s97z4x76z5q00jtt3w4w8m0000gn/T/ipykernel_33915/1496931250.py:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarities = [target_doc.similarity(doc) for doc in doc_list]


In [94]:
similarity_df

,vendor,similarity
149,NJ 07022,0.891497
147,5 FAIR Oaks Ave,0.863883
157,"Brooklyn, NY",0.850270
61,"2011 Hewitt Ave, Everett WA 98201",0.840471
111,"124 East Park Ave, Long Beach, NY 11561",0.837035
...,...,...
83,NAANCHING,0.000000
133,FourthStreetMill,0.000000
82,CHLOE81,0.000000
176,DINE_IN,0.000000
